In [26]:
import cv2
import numpy as np

In [27]:
image_as_BGR = cv2.imread("C:\\Jupyter\\pdi\\imagenes\\monedas.jpg")
image_as_GRAY = cv2.cvtColor(image_as_BGR, cv2.COLOR_BGR2GRAY)

In [30]:
valor_umbral = 0
valor_maximo = 255
tipo = cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU
return_value, image_with_treshold = cv2.threshold(image_as_GRAY, valor_umbral, valor_maximo, tipo)

kernel = np.ones((3,3), np.uint8)
image_as_opening = cv2.morphologyEx(image_with_treshold, cv2.MORPH_OPEN, kernel, iterations = 2)
background = cv2.dilate(image_as_opening, kernel, iterations = 3)
distance_transform = cv2.distanceTransform(image_as_opening,cv2.DIST_L2,5)
#print(distance_transform.flatten().tolist())
cv2.normalize(distance_transform, distance_transform,1,0,cv2.NORM_INF)
#print(distance_transform.flatten().tolist())

valor_umbral = 0.7*distance_transform.max()
valor_maximo = 255
tipo = cv2.THRESH_BINARY
return_value, foreground = cv2.threshold(distance_transform,valor_umbral,valor_maximo,tipo)
foreground = np.uint8(foreground)
image = cv2.subtract(background,foreground)

return_value,markers = cv2.connectedComponents(foreground)
print(return_value)
print(set(markers.flatten().tolist()))

markers = markers + 1
markers[image == 255] = 0
markers = cv2.watershed(image_as_BGR, markers)
min_value, max_value, min_value_coordinate, max_value_coordinate = cv2.minMaxLoc(markers)

markers = 255*(markers - min_value)/(max_value - min_value)
markers = np.uint8(markers)

markers = cv2.applyColorMap(markers, cv2.COLORMAP_JET)
cv2.imshow("Segmentación",markers)
cv2.waitKey(10000)
cv2.destroyAllWindows()

#cv2.distanceTransform(src, distanceType, maskSize[, dst]) → dst
#cv2.normalize(src[, dst[, alpha[, beta[, norm_type[, dtype[, mask]]]]]]) → dst
#cv.connectedComponents(image[, labels[, connectivity[, ltype]]]) ->retval, labels
#cv2.watershed(image, markers)
help(cv2.watershed)

25
{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24}
Help on built-in function watershed:

watershed(...)
    watershed(image, markers) -> markers
    .   @brief Performs a marker-based image segmentation using the watershed algorithm.
    .
    .   The function implements one of the variants of watershed, non-parametric marker-based segmentation
    .   algorithm, described in @cite Meyer92 .
    .
    .   Before passing the image to the function, you have to roughly outline the desired regions in the
    .   image markers with positive (\>0) indices. So, every region is represented as one or more connected
    .   components with the pixel values 1, 2, 3, and so on. Such markers can be retrieved from a binary
    .   mask using #findContours and #drawContours (see the watershed.cpp demo). The markers are "seeds" of
    .   the future image regions. All the other pixels in markers , whose relation to the outlined regions
    .   is not known an